## Семинар 1 Индекс

## Intro

### чтение файла 
- конструкция __with open__ (recommended)
- конструкция __open + close__

In [211]:
fpath = 'fpath.txt'

# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 
    
#not reccomended  
file = open(txt_fpath, 'r')  
text = file.read()    
file.close() 

### работа с файлами и папками

### os.path  
путь до файла

In [ ]:
import os

# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath('fpath.txt'))

# возвращает имя файла / папки по полному пути до него
print(os.path.basename('/your/path/to/folder/with/fpath.txt'))

# проверить существование директории - True / False
print(os.path.exists('your/path/to/any/folder/'))

### os.listdir  
возвращает список файлов в данной директории

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'
os.listdir(main_dir)

сделаем пути абсолютными, чтобы наш код не зависел от того, где лежит этот файл

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir)]

не забывайте исключать системные директории, такие как .DS_Store

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir) if not '.DS_Store' in fpath]

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'

for root, dirs, files in os.walk(main_dir):
    for name in files:
        print(os.path.join(root, name))

> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [ ]:
list(os.walk(main_dir))

##  Обратный индекс 

Сам по себе обратный индекс не может осуществлять поиск, для этого необходимо добавить к нему определенную метрику. Это не совсем очевидная задача, поэтому немного отложим ее. А сейчас посмотрим, что полезного можно вытащить из индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе. Так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. какой документ является самым большим / маленьким (очень изощренный способ, когда есть _len_)

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

### __Задача__: 
получите обратный индекс для коллекция документов.    
Перед этим постройте матрицу терм-документ и сделайте функцию булева поиска, которая по запросу будет возвращать 5 релевантных документов.   
В качестве коллекции возьмите сценарий сезонов сериала Друзья. Одна серия - один документ.

[download_friends_corpus](https://yadi.sk/d/k_M7n63A3adGSz)

In [2]:
import string
from nltk.corpus import stopwords
from string import punctuation

Этапы:   
    1. получить коллекцию документов   --- +
    2. для каждого файла коллекции сделать необходимую на ваш взгляд предобработку --- +
    3. получить матрицу терм-документ, написать функцию поиска по ней --- +
    4. получить обратный индекс в виде словаря, где ключ - нормализованное слово,
    значение - список файлов, в которых это слово встречается  --- +
    5. вывести кусочек индекса в виде таблицы --- +
    6. сделать анализ обратного индекса. Это задание принимается в виде кода и ответов на вопросы --- +

In [5]:
main_dir = '/Users/Stoneberry/Downloads/Friends/'

In [6]:
stops = stopwords.words('russian')

In [7]:
from collections import defaultdict

In [8]:
import re

def remove(text):
    new = re.sub(r'\ufeff', '', text)
    new = re.sub(r'\n', ' ', new)
    new = re.sub(r'www.tvsubtitles.net', '', new)
    new = re.sub(r'([\W]|\d)', ' ', new)
    new = re.sub('(\n|\t|\v|\r)', ' ', new)
   # new = re.sub(r'[ \t\v\r\n\f]', ' ', new)
    return new

In [37]:
all_ = defaultdict(str)

In [38]:
idx = defaultdict(str)

In [39]:
from pymystem3 import Mystem

mystem = Mystem()

Словарь, чтобы потом понять сколько серия в каких сезонах

In [40]:
seasons = defaultdict(list)

Средняя длина текста (приблизительно)

In [41]:
avr = []

In [42]:
i = 0
l = 0
mystem = Mystem()

for root, dirs, files in os.walk(main_dir):
    for name in files:
        if not '.DS_Store' in name:
            with open(os.path.join(root, name), 'r') as f:
                text = f.read() 
                a = mystem.lemmatize(remove(text))
                avr.append(len(a))
                all_[name[:-7]] = ''.join(a)
                if i == 165:
                    break
                idx[i] = name[:-7]
                seasons[l].append(name[:-7])
                i += 1
    l += 1

Пример текста

In [15]:
print(all_[name[:-7]][0:200])

боже мой   я знать  у моника быть ребенок   это можно считать за  что то новенький    бог ты мой   думать  поэтому чендлер сбегать   должно быть  она проходить тест совсем недавно  я мусор вчера вечер


In [16]:
print(all_[name[:-7]][-200:])

ребенок   у мы быть ребенок   фиби находить твой тест на беременность   я не делать тест на беременность   тогда кто делать   посмотреть на они   у они быть ребенок                                   



In [17]:
len(all_)

165

Tf-idf матрица

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [19]:
vect = TfidfVectorizer()
x = vect.fit_transform(list(all_.values())) 

In [20]:
data = pd.DataFrame(x.todense()).transpose()

In [21]:
data.columns = list(idx.values())

In [22]:
data.shape

(14113, 165)

In [23]:
data['Term'] = vect.get_feature_names()

In [24]:
data = data.set_index(['Term'])

В качестве частоты будем использовать сумму по строчке.

In [25]:
data['freq'] = data.sum(axis=1)

In [44]:
data.head()

,Friends - 1x01 - The One Where Monica Gets A Roommate,Friends - 1x02 - The One With The Sonogram At The End,Friends - 1x03 - The One With The Thumb,Friends - 1x04 - The One With George Stephanopoulos,Friends - 1x05 - The One With The East German Laundry Detergent,Friends - 1x06 - The One With The Butt,Friends - 1x07 - The One With The Blackout,Friends - 1x08 - The One Where Nana Dies Twice,Friends - 1x09 - The One Where Underdog Gets Away,Friends - 1x10 - The One With The Monkey,...,Friends - 7x16 - The One With The Truth About London,Friends - 7x17 - The One With The Cheap Wedding Dress,Friends - 7x18 - The One With Joey's Award,Friends - 7x19 - The One With Ross And Monica's Cousin,Friends - 7x20 - The One With Rachel's Big Kiss,Friends - 7x21 - The One With The Vows,Friends - 7x22 - The One With Chandler's Dad,Friends - 7x23 - The One With Chandler And Monica's Wedding (1),Friends - 7x24-25 - The One With Chandler And Monica's Wedding (2),freq
Term,,,,,,,,,,,,,,,,,,,,,
after,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025744
again,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025580
ahh,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024608
all,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068349
and,0.0,0.0,0.0,0.053245,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076152


In [25]:
type(data)

pandas.core.frame.DataFrame

Функция поиска

In [58]:
def data_search(data, word, episode=None, non=False):

    if not isinstance(word, str):
        raise ValueError
    if word == '':
        return None
    
    try:
        data.loc[word]
    except KeyError:
        raise ValueError('There is no such word')

    if episode is None:
        answer = set()
        if not non: 
            for index, item in enumerate(data.loc[word].drop('freq')):
                if item > 0:
                    answer.add(idx[index])
        else:
            for index, item in enumerate(data.loc[word]):
                if item == 0:
                    answer.add(idx[index])
        return answer

    if not isinstance(episode, str):
        raise ValueError

    return data.loc[word][episode]

In [59]:
data_search(data, 'after')

{'Friends - 3x07 - The One With The Race Car Bed'}

In [60]:
data_search(data, 'and')

{'Friends - 1x04 - The One With George Stephanopoulos',
 'Friends - 5x16 - The One With The Cop'}

In [61]:
data_search(data, 'моника', non=True)

{'Friends - 1x11 - The One With Mrs. Bing',
 'Friends - 1x16 - The One With Two Parts (1)',
 "Friends - 2x01 - The One With Ross's New Girlfriend.DVDRip",
 'Friends - 2x02 - The One With The Breast Milk',
 "Friends - 2x24 - The One With Barry And Mindy's Wedding",
 'Friends - 3x03 - The One With The Jam',
 'Friends - 3x07 - The One With The Race Car Bed',
 "Friends - 3x11 - The One Where Chandler Can't Remember Which Sister",
 'Friends - 3x20 - The One With The Dollhouse',
 'Friends - 4x16 - The One With The Fake Party',
 "Friends - 5x12 - The One With Chandler's Work Laugh",
 'Friends - 5x16 - The One With The Cop',
 'Friends - 5x18 - The One Where Rachel Smokes',
 'Friends - 5x21 - The One With The Ball',
 'Friends - 5x23-24 - The One In Vegas (1)',
 'Friends - 6x09 - The One Where Ross Got High',
 'Friends - 6x25-26 - The One With The Proposal (2)',
 'Friends - 7x10 - The One With The Holiday Armadillo',
 'Friends - 7x13 - The One Where Rosita Dies',
 'Friends - 7x21 - The One With 

Обратный индекс

![](Friends/wedding.png)

Напоминание:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, старайтесь этого избегать

In [26]:
main_dir = '/your/path/with/Frinds/collection/'
files_list = []

### пройдитесь по всем папкам коллекции и соберите все пути .txt файлов
### _check : в коллекции должно быть 165 файлов


In [22]:
### постройте матрицу терм-документ
term_doc_matrix = []


### напишите функцию булева поиска по построенной матрице
def boolean_search(data, text):
    """
    Produces a Boolean search according with the term-document matrix
    :return: list of first 5 relevant documents
    """

#запросы 
input_text = [
    'Моника & Фиби & Рэйчел & Чендлер & Джои & Росс',
    '(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джои) & Росс', 
    '(НЕ Моника) & Фиби & Рэйчел & Чендлер & Джои & (НЕ Росс)'
]

In [238]:
#text ='Моника & Фиби & Рэйчел & Чендлер & Джоуи & Росс'
text = '(НЕ Моника) & Фиби & Рэйчел & Чендлер & Джоуи & (НЕ Росс)'
#text = 'after ИЛИ and'
#text = 'after ИЛИ and ИЛИ ahh'
#text = 'and & (НЕ Моника)'
text = '(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джоуи) & Росс'
# не (моника или чедлер)
#boolean_search(data, text)

In [66]:
def boolean_search(data, text):
    """
    Produces a Boolean search according with the term-document matrix
    :return: list of first 5 relevant documents
    
    
    AND = Пересечение - &
    OR = Объединение - ИЛИ
    NOT = Исключение - НЕ
    """
    if not isinstance(text, (str, list)):
        raise ValueError
    
    i = 0 
    j = 1

    names = text.split(' ')
    answer = []
    operand = ['&', 'ИЛИ', 'НЕ', '(НЕ']
    block = []

    ands = False
    ors = False
    nons = False

    par = False

    bl_ands = False
    bl_ors = False
    bl_non = False
    bl_end = False

    def plus(answer, cur):
        answer.append(answer[-1] & cur)

    def or_(answer, cur):
        answer.append(answer[-1] | cur)

    
    for index, item in enumerate(names):
    
        if item != '' and item not in operand:
        
            if item.startswith('('):
                item = item[1:]
                block.append(item)
                par = True
        
            if item.endswith(')'):
                item = item[:-1]
                bl_end = True
        
            if par:
                if bl_non:
                    cur = data_search(data, item.lower(), non=True)
                    nons = False
                else:
                    cur = data_search(data, item.lower())
    
                if bl_ands:
                    plus(block, cur)
                    bl_ands = False
            
                if bl_ors:
                    or_(block, cur)
                    bl_ors = False
            
                else:
                    block.append(cur)

                if bl_end:
                    if ands:
                        plus(answer, block[-1])
                        ands = False
                    if ors:
                        or_(answer, block[-1])
                        ors = False
                    else:
                        answer.append(block[-1])
                    par = False
                
            else:
                if nons and item[0] != '(':
                    cur = data_search(data, item.lower(), non=True)
                    nons = False
                elif not nons:
                    cur = data_search(data, item.lower())

                if index == 0:
                    answer.append(cur)
            
                if ands:
                    plus(answer, cur)
                    ands = False
                if ors:
                    or_(answer, cur)
                    ors = False
    
        elif item == '&':
            if par:
                bl_ands = True
            if len(answer) > 0:
                ands = True
            else:
                raise ValueError('Enter correct string')
    
        elif item == 'ИЛИ':
            if par:
                bl_ors = True
            elif len(answer) > 0:
                ors = True
            else:
                raise ValueError('Enter correct string')
    
        elif item == 'НЕ':
            if par:
                bl_non = True
            else:
                nons = True

        elif item == '(НЕ':
            bl_non = True
            par = True
    
    return [itm for ind, itm in enumerate(answer[-1]) if ind < 5]

In [67]:
text = '(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джоуи) & Росс'
boolean_search(data, text)

['Friends - 3x01 - The One With The Princess Leia Fantasy',
 'Friends - 4x01 - The One With The Jellyfish.Tv',
 "Friends - 7x18 - The One With Joey's Award",
 'Friends - 7x05 - The One With The Engagement Picture',
 "Friends - 4x03 - The One With The 'Cuffs"]

<img src="pics/inv_index3.svg" alt="Drawing" style="width: 600px;"/>

Совет для построения обратного индекса: 
> В качестве словаря используйте ``` defaultdict ``` из модуля collections   
Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [69]:
def inverted_index(vect) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    
    d = defaultdict(list)

    for word in vect.get_feature_names():
        d[word] = [i for i in data_search(data, word)]
    
    return d

In [71]:
d = inverted_index(vect)

С помощью обратного индекса произведите следующую аналитику:  

1) общая аналитика
- какое слово является самым частотным?
- какое самым редким?
- какой набор слов есть во всех документах коллекции?

2) частота встречаемости имен главных героев в каждом сезоне      
- какой сезон был самым популярным у Чендлера? у Моники?   
- кто из главных героев статистически самый популярный? 


Таблица индекса

In [72]:
import collections

In [73]:
inv_ind = pd.DataFrame({'term': vect.get_feature_names(),
                        'docs': list(d.values()),
                        'freq': data['freq']
                       })

In [74]:
inv_ind['lenght'] = inv_ind.docs.map(lambda x: len(x))

In [75]:
inv_ind.head()

,docs,freq,term,lenght
Term,,,,
after,[Friends - 4x03 - The One With The 'Cuffs],0.025744,after,1
again,[Friends - 1x15 - The One With The Stoned Guy],0.025580,again,1
ahh,[Friends - 7x11 - The One With All The Cheesec...,0.024608,ahh,2
all,[Friends - 2x02 - The One With The Breast Milk...,0.068349,all,2
and,"[Friends - 3x05 - The One With Frank Jr., Frie...",0.076152,and,2


Какое слово является самым частотным?

In [76]:
inv_ind.ix[inv_ind['freq'].idxmax()]

docs      [Friends - 7x05 - The One With The Engagement ...
freq                                                 55.332
term                                                     ты
lenght                                                    1
Name: ты, dtype: object

Какое самым редким?

In [77]:
inv_ind.ix[inv_ind['freq'].idxmin()]

docs      [Friends - 6x04 - The One Where Joey Loses His...
freq                                              0.0156364
term                                                     би
lenght                                                    1
Name: би, dtype: object

Какой набор слов есть во всех документах коллекции?

In [78]:
list(inv_ind.loc[inv_ind['lenght'] == 165].term)

['акула',
 'амузбушнуло',
 'вбрасывать',
 'век',
 'внук',
 'гигиенистка',
 'джоуевский',
 'домоводческий',
 'зерно',
 'кинзовый',
 'кнопка',
 'концентрический',
 'космолагерь',
 'кроче',
 'кружиться',
 'легальный',
 'наступать',
 'начистоту',
 'официанточка',
 'подкатывать',
 'премия',
 'раздвигать',
 'речь',
 'справляться',
 'требовать',
 'уберегать',
 'указание',
 'формально',
 'хота',
 'шведский',
 'шухер',
 'энн']

Какой сезон был самым популярным у Чендлера? у Моники?

Чендлер

In [79]:
def popular(name):

    row = data.loc[name.lower()]
    row = row.drop('freq')

    maximum = 0
    seas = ''

    for i in range(1, 8):
        a = row[seasons[i]].sum()
        if maximum < a:
            maximum = a
            seas = i

    return seas

In [80]:
popular('чендлер')

2

Моника

In [81]:
popular('моника')

4

Кто из главных героев статистически самый популярный?

In [82]:
data.loc[['моника', 'росс', 'джоуи', 'чендлер', 'фиби', 'рэйчел']]['freq']

Term
моника     3.881715
росс       5.182460
джоуи      3.999995
чендлер    4.085088
фиби       3.236180
рэйчел     2.427932
Name: freq, dtype: float64

Росс самый популярный

## Функция ранжирования Okapi BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k_1+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где   
>$f(q_i,D)$ - частота слова $q_i$ в документе $D$ (TF)       
$|D|$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [83]:
from math import log

In [84]:
def idf(N, q):
    n = inv_ind.loc[q]['lenght'] 
    a = N - n + 0.5
    return log(a / n + 0.5)

In [85]:
### реализуйте эту функцию ранжирования 
from math import log

k1 = 2.0
b = 0.75
N = 165
avgdl = np.mean(avr)
opr = [' ', '  ', '\t', '\n']
n = 0

def score_BM25(qf, dl, avgdl, k1, b, N, n, data):
    """
    Compute similarity score between search query and documents from collection
    :return: score
    
    qf - запрос
    dl - название документа = название серии

    """
    if not isinstance(qf, str):
        raise ValueError
    if not isinstance(dl, str):
        raise ValueError

    words = mystem.lemmatize(remove(qf))
    words = [word for word in words if word not in opr]
    
    ans = 0
    
    for word in words:
        if word in data.index and dl in data.columns:
            f = (data.loc[word][dl] / idf(N, word))
            a = (k1 + 1) * f
            b = f + k1*(1 - b + b*(len(words)) / avgdl)
            ans += (a / b) * idf(N, word)
    

    return ans

In [86]:
score_BM25('моника и чендлер', 'Friends - 5x16 - The One With The Cop', avgdl, k1, b, N, n, data)

0.0778204997492204

### __Задача__:    
напишите функцию, которая сортирует поисковую выдачу для любого входящего запроса согласно метрике *Okapi BM25*.    
Выведите 10 первых результатов и их скор по запросу **рождественские каникулы**. 

In [87]:
def compute_sim(text, doc, data):
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    return score_BM25(text, doc, avgdl, k1, b, N, n, data)


def get_search_result(text, data):
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    
    if not isinstance(text, str):
        raise ValueError
    
    arr = []
    
    for doc in data.columns:
        arr.append((doc, compute_sim(text, doc, data)))
    
    return arr

In [88]:
sorted(get_search_result('рождественские каникулы', data),
       key=lambda x: x[1], reverse=True)[1:11]

[("Friends - 7x19 - The One With Ross And Monica's Cousin",
  0.6961737811621609),
 ('Friends - 6x01 - The One After Vegas', 0.508830209803203),
 ('Friends - 5x16 - The One With The Cop', 0.36194261665203264),
 ("Friends - 6x19 - The One With Joey's Fridge", 0.33587383556912054),
 ('Friends - 1x02 - The One With The Sonogram At The End',
  0.23735908076155765),
 ('Friends - 5x21 - The One With The Ball', 0.22718732672693115),
 ('Friends - 1x08 - The One Where Nana Dies Twice', 0.22223873275445033),
 ('Friends - 3x18 - The One With The Hypnosis Tape', 0.2119813698252759),
 ('Friends - 1x09 - The One Where Underdog Gets Away', 0.1158119538177487),
 ("Friends - 5x13 - The One With Joey's Bag", 0.11159121975604475)]